In [149]:
import pandas as pd
# 과학적 표기 안하도록
pd.options.display.float_format = '{:.2f}'.format

import numpy as np
import datetime

---
### 증감 결과 확인을 위한 함수

In [136]:
def increasing(col):
    for i in range(len(col)):
        if(i == 0):
            temp = col[i]
        else:
            if(temp > col[i]):
                return 0
            temp = col[i]
    return 1 

def decreasing(col):
    for i in range(len(col)):
        if(i==0):
            temp = col[i]
        else:
            if(temp < col[i]):
                return 0
            temp = col[i]
    return 1

---
### selenium으로 크롤링


In [222]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs

driver = webdriver.Chrome("/Users/naver/Downloads/chromedriver")

In [3]:
# code로 접근 가능한 url
code_url = "https://finance.naver.com/item/main.nhn?code="
# 종목 분석 url
coinfo_url = "https://finance.naver.com/item/coinfo.nhn?code="
# 종목 분석 url (!naver)
coinfo_url_2 = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=%s&cn="
# 시가총액 목록 url
size_market_url = "https://finance.naver.com/sise/sise_market_sum.nhn?&page="

In [292]:
# 오늘의 시가총액 code 목록 구하기 (50 * 5)
# codeL = []
# for i in range(1,6):
#     driver.get(size_market_url+str(i))
#     tempL = driver.find_elements_by_css_selector("#contentarea > div.box_type_l > table.type_2 > tbody > tr > td.center > a")
#     tempL = [x.get_attribute("href").split("code=")[1] for x in tempL]
#     codeL += tempL

# 컨센서스 없는 종목 제외
removeL = ["005935","069500","088980", "102110", "005387"]
for rem in removeL:
    codeL.remove(rem)
    

In [306]:
# 오늘의 시가총액 code 목록 저장 
# todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
# f = open("./"+todayDate+"_시가총액_상위권.txt", 'w')
# f.write("\n".join(codeL))
# f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = "2020_Feb_10"
f = open("./"+useDate+"_시가총액_상위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

### 컨센서스 확인 함수

In [158]:
# 삼성전자 확인
driver.get(coinfo_url_2.replace("%s", codeL[0]))

In [214]:
coinfo_url_2.replace("%s", codeL[0])

'https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=005930&cn='

In [223]:
soup = bs(requests.get(coinfo_url_2.replace("%s", codeL[0])).text, 'html.parser')

In [224]:
soup.select("#pArea > div.wrapper-table > div > table > tbody > tr:nth-child(1) > td > dl > dt:nth-child(1) > span")

[]

In [226]:
coinfo_url_2.replace("%s", codeL[0])

'https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=005930&cn='

In [230]:
consenUrl = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&cmp_cd=005930&finGubun=MAIN&frq=0&sDT=20200207&chartType=svg"

In [233]:
pd.DataFrame(requests.get(consenUrl).json()["JsonData"])

,YYMM,SALES,YOY,OP,NP,EPS,BPS,PER,PBR,ROE,EV,MAIN,TOT_ROW
0,2015.12(A),912.8,,"-2,036.4","19,049.5","38,489","50,326",,,107.38,,IFRS별도,7
1,2016.12(A),"2,946.2",222.77,-304.2,"-1,768.2","-3,113","61,700",-48.50,2.45,-5.16,324.07,IFRS별도,7
2,2017.12(A),"4,646.3",57.70,659.8,-969.7,"-1,466","60,099",-253.14,6.17,-2.41,182.00,IFRS별도,7
3,2018.12(A),"5,358.1",15.32,556.8,"2,241.1","3,387","62,805",114.11,6.15,5.51,172.21,IFRS별도,7
4,2019.12(E),"5,983.5",11.67,298.1,454.3,687,"63,319",636.44,6.90,1.09,184.29,IFRS별도,7
5,2020.12(E),"9,318.7",55.74,"1,979.6","2,366.2","3,576","68,265",145.13,7.60,5.44,104.20,IFRS별도,7
6,2021.12(E),"12,070.0",29.52,"3,327.4","3,717.9","5,619","73,437",92.36,7.07,7.93,73.71,IFRS별도,7


In [244]:
soup = bs(requests.get(code_url + "005930").text, 'html.parser')
name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
name

'삼성전자'

In [250]:
def Consensus_selenium(code):
    driver.get(coinfo_url_2.replace("%s", code))
    
    # 종목 이름
    name = driver.find_element_by_css_selector("#pArea > div.wrapper-table > div > table > tbody > tr:nth-child(1) > td > dl > dt:nth-child(1) > span").text
    
    # 컨센서스 테이블 구축
    table = driver.find_element_by_css_selector("#cTB511")
    tableRow = table.find_element_by_css_selector("tbody")
    columnL = table.find_element_by_css_selector("thead").text.replace("\n","").split(" ")
    thL = [x.find_element_by_css_selector("th").text for x in tableRow.find_elements_by_css_selector("tr")]
    trL = [["0" if y.text =="" else y.text for y in x.find_elements_by_css_selector("td")] for x in tableRow.find_elements_by_css_selector("tr")]
    for idx in range(len(trL)):
        trL[idx].insert(0,thL[idx])
    df_table = pd.DataFrame(trL, columns = columnL)

    # 매출액 ~ EV/EBITDA 숫자형으로 변경
    df_table.iloc[:, 1:-1] = df_table.iloc[:,1:-1].applymap(lambda x: float(x.replace(",","")))
    
    # 증감 결과 df table로 확인
    result_row = [0]*12
    for inc in [1, 2, 3, 4, 5, 6, 9]:
        if(increasing(df_table.iloc[:, inc])):
            result_row[inc] = 1
    for dec in [7, 8, 10]:
        if(decreasing(df_table.iloc[:, dec])):
            result_row[dec] = 1

    result_df = pd.DataFrame([result_row], columns = df_table.columns)
    df_table.append(result_df, ignore_index = True)
    
     #증감 결과 매출액 / 영업이익 / per * 10 roe * 5
    formula = [0,10,1,10,1,1,1,10,1,5,1,0]
    score = np.dot(result_row,formula)
    
    print(code,name, score)
    if(score > 20):
        print(df_table.iloc[:, 1:-1])

In [290]:
# 종목 이름 얻기
code = "005387"
soup = bs(requests.get(code_url + code).text, 'html.parser')
name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
name

'현대차2우B'

In [319]:
def Consensus(code):
    code = str(code)
        
    # 종목 이름 얻기
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    
    # 컨센서스 테이블 구축
    consenUrl = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&cmp_cd="\
                + code + "&finGubun=MAIN&frq=0&chartType=svg"
    if(requests.get(consenUrl).json()["JsonData"] == []):
        return 
    df_table = pd.DataFrame(requests.get(consenUrl).json()["JsonData"]).iloc[:, :-1]

    # 매출액 ~ EV/EBITDA 숫자형으로 변경
    df_table.iloc[:, 1:-1] = df_table.iloc[:,1:-1].applymap(lambda x: "0" if x == "" else x).applymap(lambda x: float(x.replace(",","")))
    
    # 증감 결과 df table로 확인
    result_row = [0]*12
    for inc in [1, 2, 3, 4, 5, 6, 9]:
        if(increasing(df_table.iloc[:, inc])):
            result_row[inc] = 1
    for dec in [7, 8, 10]:
        if(decreasing(df_table.iloc[:, dec])):
            result_row[dec] = 1

    result_df = pd.DataFrame([result_row], columns = df_table.columns)
    df_table.append(result_df, ignore_index = True)
    
     #증감 결과 매출액 / 영업이익 / per * 10 roe * 5
    formula = [0,10,1,10,1,1,1,10,1,5,1,0]
    score = np.dot(result_row,formula)
    
    print(code, name, score)
#     if(score > 20):
#         print(code, name, score)
#         print(df_table.iloc[:, 1:-1])

---

## 시가총액 상위 50개의 컨센서스 확인

In [320]:
Consensus("039130")

039130 하나투어 0


In [321]:
for i in range(250):
    Consensus(codeL[i])

005930 삼성전자 1
000660 SK하이닉스 1
207940 삼성바이오로직스 10
035420 NAVER 11
005380 현대차 10
051910 LG화학 11
068270 셀트리온 11
028260 삼성물산 0
006400 삼성SDI 11
012330 현대모비스 1
051900 LG생활건강 23
005490 POSCO 0
017670 SK텔레콤 0
055550 신한지주 1
105560 KB금융 1
034730 SK 1
000270 기아차 11
015760 한국전력 1
018260 삼성에스디에스 23
035720 카카오 11
036570 엔씨소프트 0
032830 삼성생명 1
003550 LG 1
033780 KT&G 1
096770 SK이노베이션 1
066570 LG전자 1
090430 아모레퍼시픽 2
086790 하나금융지주 12
009150 삼성전기 0
000810 삼성화재 3
010950 S-Oil 0
009540 한국조선해양 0
010130 고려아연 1
251270 넷마블 1
316140 우리금융지주 14
011170 롯데케미칼 1
030200 KT 13
021240 웅진코웨이 0
032640 LG유플러스 1
024110 기업은행 11
034220 LG디스플레이 0
035250 강원랜드 2
018880 한온시스템 1
002790 아모레G 0


KeyboardInterrupt: 